# Eat Safe, Love

## Notebook Set Up

In [17]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [46]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [47]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [48]:
# review the collections in our database
print (db.list_collection_names())

['establishments']


In [49]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [50]:
# Find the establishments with a hygiene score of 20
query = { 'scores.Hygiene': {'$eq': 20} }

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(establishments.find_one(query))



0
None


In [51]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(len(hygiene_df))

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)


0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [52]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = { 'RatingValue': { '$gte': 4 }, 'LocalAuthorityName': {'$regex': 'London'}}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(establishments.find_one(query))


0
None


In [53]:
# Convert the result to a Pandas DataFrame
ratings_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(len(ratings_df))

# Display the first 10 rows of the DataFrame
ratings_df.head(10)


0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [54]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# get the Penang Flavors document
establishment_coordinates = establishments.find_one({'BusinessName':'Penang Flavours'})

degree_search = 0.01

# converting the coordinates to string and then to float so I can perform arthmetic.
# without doing this, would receive an error that I couldn't add Decimal128 and float
latitude = float(str(establishment_coordinates['geocode']['latitude']))
longitude = float(str(establishment_coordinates['geocode']['longitude']))

# find documents where RatingValue is 5 and latitude and longitude are +- 0.01 degrees of Penang Flavours coordinates
query = {
    'RatingValue': 5,
    'geocode.latitude': { '$gt': latitude - degree_search, '$lt': latitude + degree_search },
    'geocode.longitude': { '$gt': longitude - degree_search, '$lt': longitude + degree_search}
}

# order by hygiene
sort = [('scores.Hygiene', 1)] 

# find the top 5 results
top_5_results = list(establishments.find(query).sort(sort).limit(5))

# Print the results
pprint(top_5_results)

[]


In [55]:
# Convert result to Pandas DataFrame
top_5_df = pd.DataFrame(top_5_results)
top_5_df


""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [56]:
# Create a pipeline that: 

# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene': 0}}

# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}

# 3. Sorts the matches from highest to lowest
sort_query = { '$sort': { 'count': -1 }}

pipeline = [match_query, group_query, sort_query]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f'Number of Local Authorities with establishments that have a hygiene score of 0: {len(results)}')

# Print the first 10 results
pprint(results[0:10])

Number of Local Authorities with establishments that have a hygiene score of 0: 0
[]


In [57]:
# Convert the result to a Pandas DataFrame
hygiene_zero_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(len(hygiene_zero_df))

# Display the first 10 rows of the DataFrame
hygiene_zero_df.head(10)

0


""
